# 카메라 스티커앱을 개선하자

## 프로젝트 (1) moviepy로 비디오 처리하기

In [1]:
from moviepy.editor import VideoClip, VideoFileClip
from moviepy.editor import ipython_display
import cv2
import numpy as np
import os

### 1. moviepy를 이용해서 주피터 노트북 상에서 비디오를 읽고 쓰는 프로그램을 작성한다.

In [2]:
# 읽기
video_path = './images/video2.mp4'
clip = VideoFileClip(video_path)
clip = clip.resize(width=640)
clip.ipython_display(fps=30, loop=True, autoplay=True, rd_kwargs=dict(logger=None))

# 쓰기
result_video_path = './images/mvpyresult.mp4'
clip.write_videofile(result_video_path)

t:   0%|          | 0/404 [00:00<?, ?it/s, now=None]                

Moviepy - Building video ./images/mvpyresult.mp4.
MoviePy - Writing audio in mvpyresultTEMP_MPY_wvf_snd.mp3
MoviePy - Done.
Moviepy - Writing video ./images/mvpyresult.mp4



Moviepy - Done !
Moviepy - video ready ./images/mvpyresult.mp4


### 2. moviepy 로 읽은 동영상을 numpy 형태로 변환하고 영상 밝기를 50% 어둡게 만든 후에 저장한다.

In [3]:
# 읽기
video_path = './images/video2.mp4'
clip = VideoFileClip(video_path)
clip = clip.resize(width=640)
clip.ipython_display(fps=30, loop=True, autoplay=True, rd_kwargs=dict(logger=None))

# clip 에서 numpy 로 데이터 추출
vlen = int(clip.duration*clip.fps)
video_container = np.zeros((vlen, clip.size[1], clip.size[0], 3), dtype=np.uint8)
for i in range(vlen):
    img = clip.get_frame(i/clip.fps)
    video_container[i] = (img * 0.5).astype(np.uint8)

# 새 clip 만들기
dur = vlen / clip.fps
outclip = VideoClip(lambda t: video_container[int(round(t*clip.fps))], duration=dur)

# 쓰기
result_video_path2 = './images/mvpyresult2.mp4'
outclip.write_videofile(result_video_path2, fps=30)

t:  12%|█▏        | 49/403 [00:00<00:00, 484.26it/s, now=None]

Moviepy - Building video ./images/mvpyresult2.mp4.
Moviepy - Writing video ./images/mvpyresult2.mp4



Moviepy - Done !
Moviepy - video ready ./images/mvpyresult2.mp4


### 3. 영상을 읽고 쓰는 시간을 측정한다. OpenCV를 사용할 때와 차이를 측정한다.

In [4]:
# CASE 1 : moviepy 사용
start = cv2.getTickCount()
clip = VideoFileClip(video_path)
clip = clip.resize(width=640)

vlen = int(clip.duration*clip.fps)
video_container = np.zeros((vlen, clip.size[1], clip.size[0], 3), dtype=np.uint8)

for i in range(vlen):
    img = clip.get_frame(i/clip.fps)
    video_container[i] = (img * 0.5).astype(np.uint8)

dur = vlen / clip.fps
outclip = VideoClip(lambda t: video_container[int(round(t*clip.fps))], duration=dur)

mvpy_video_path = './images/mvpyresult.mp4'
outclip.write_videofile(mvpy_video_path, fps=30)

time = (cv2.getTickCount() - start) / cv2.getTickFrequency()
print (f'[INFO] moviepy time : {time:.2f}ms')

t:  12%|█▏        | 49/403 [00:00<00:00, 486.34it/s, now=None]

Moviepy - Building video ./images/mvpyresult.mp4.
Moviepy - Writing video ./images/mvpyresult.mp4



Moviepy - Done !
Moviepy - video ready ./images/mvpyresult.mp4
[INFO] moviepy time : 2.65ms


In [5]:
# CASE 2 : OpenCV 사용
start = cv2.getTickCount()
vc = cv2.VideoCapture(video_path)

cv_video_path = './images/cvresult.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
vw = cv2.VideoWriter(cv_video_path, fourcc, 30, (640,360))

vlen = int(vc.get(cv2.CAP_PROP_FRAME_COUNT))

for i in range(vlen):
    ret, img = vc.read()
    if ret == False: break
    
    img_result = cv2.resize(img, (640, 360)) * 0.5
    vw.write(img_result.astype(np.uint8))
    
time = (cv2.getTickCount() - start) / cv2.getTickFrequency()
print (f'[INFO] cv time : {time:.2f}ms')

[INFO] cv time : 1.34ms


### 4. moviepy 를 이용할 때의 장단점을 분석한다. 주피터 노트북에 답변을 작성해 코드와 함께 제출한다.

- OpenCV의 읽기, 쓰기 속도가 moviepy에 비해 두 배 정도 빠른 것 같다.
- OpenCV는 웹캠을 이용하여 실시간 Inference가 가능하지만 moviepy는 저장된 영상에만 적용 가능하다.
- 저장된 영상을 다루는 데에는 moviepy쪽이 더 간편한 것 같다.

## 프로젝트 (2) 어디까지 만들고 싶은지 정의하기

### 1. 실시간 카메라 스티커앱을 만들어본다.

#webcam_sticker.py

```python
import numpy as np
import cv2
import dlib

from newaddsticker import img2sticker

detector_hog = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor('./models/shape_predictor_68_face_landmarks.dat')

def main():
    cv2.namedWindow('show', 0)
    cv2.resizeWindow('show', 640, 360)

    vc = cv2.VideoCapture(0)   # 연결된 영상 장치의 인덱스, 하나만 있는 경우 0을 사용
    img_sticker = cv2.imread('./images/king.png')

    vlen = int(vc.get(cv2.CAP_PROP_FRAME_COUNT))
    print (vlen) # 웹캠은 video length 가 0 입니다.

    # 정해진 길이가 없기 때문에 while 을 주로 사용합니다.
    # for i in range(vlen):
    while True:
        ret, img = vc.read()
        if ret == False:
            break
        start = cv2.getTickCount()
        img = cv2.flip(img, 1)  # 보통 웹캠은 좌우 반전

        # 스티커 메소드를 사용
        img_result = img2sticker(img, img_sticker.copy(), detector_hog, landmark_predictor)   

        time = (cv2.getTickCount() - start) / cv2.getTickFrequency() * 1000
        print ('[INFO] time: %.2fms'%time)

        cv2.imshow('show', img_result)
        key = cv2.waitKey(1)
        if key == 27:
            break


if __name__ == '__main__':
    main()
```

### 2. 스티커앱을 실행하고 카메라를 고정하고 서서히 멀어져본다. 혹은 아주 가까이 다가가 본다. 얼굴을 찾지 못하는 거리를 기록한다.

- 실제로 거리를 측정해본 결과 광량이 충분해서 얼굴의 윤곽만 잘 알아볼 수 있다면 최소 30cm에서 최대 250cm 이상에서도 인식이 가능했다. 방 크기의 한계로 250cm 까지 밖에 테스트하지 못했다.

### 3. 다시 자리로 돌아온 후 고개를 상하좌우로 움직인다. yaw, pitch, roll 각도의 개념을 직접 실험해 보고 각각 몇 도까지 정상적으로 스티커앱이 동작하는지 기록한다.

- yaw : -45 ~ 45도
- pitch : -20 ~ 30도
- roll : -60 ~ 60도
- 실제로 측정해보니 yaw와 roll방향은 그래도 어느 정도 움직여도 잘 인식하였으나 pitch방향은 조금만 틀어져도 동작하지 않았다.

### 4. 만들고 싶은 스티커앱의 스펙(허용 거리, 허용 인원 수, 허용 각도, 안정성)을 정한다.

- 거리 : 50cm ~ 150cm -> 일반적으로 셀카를 찍을 때 팔을 뻗어서 찍기 때문에 팔을 뻗었을 때와 얼굴 사이의 거리인 50cm 정도부터 여럿이 찍을 때를 대비하여 150cm 까지 설정
- 인원 : 최대 2인 -> 커플 또는 친구와 2인으로 찍을 때 가장 적절
- 허용 각도 : yaw : -50 ~ 50도, roll: -45 ~ 45도, pitch -45 ~ 45도 -> 일반적으로 많이 찍는 정면 이외에도 일명 얼짱 각도라고 불리는 대각선 위에서 찍는 각도 및 눈 코 입이 전부 보이는 각도
- 안정성 : 위 조건을 만족하면서 흔들림 없이 편안하게 칼만 필터를 이용하여 noise에 영향을 적게 받도록 한다.

## 프로젝트 (3) 스티커 Out Bound 예외처리하기

### 1. 지금까지 만든 스티커앱을 이용해서 예외 상황을 찾는다. 특히 서서히 영상에서 좌우 경계 밖으로 나가며 코드의 행동을 확인한다.

```python
Traceback (most recent call last):
  File "webcam_sticker.py", line 42, in <module>
    main()
  File "webcam_sticker.py", line 30, in main
    img_result = img2sticker(img, img_sticker.copy(), detector_hog, landmark_predictor)   
  File "/home/ssac10/aiffel/video_sticker_app/newaddsticker.py", line 48, in img2sticker
    cv2.addWeighted(sticker_area, 1.0, img_sticker, 0.7, 0)
cv2.error: OpenCV(4.5.1) /tmp/pip-req-build-hj027r8z/opencv/modules/core/src/arithm.cpp:666: error: (-209:Sizes of input arguments do not match) The operation is neither 'array op array' (where arrays have the same size and the same number of channels), nor 'array op scalar', nor 'scalar op array' in function 'arithm_op'
```

- 위와 같은 오류가 발생하며 프로그램이 종료된다.
- 어쩐지 위에서부터 계속 위와 같은 오류로 프로그램이 종료되어 어딘가 코드가 문제가 있을 것이라고 생각했다.

### 2. 문제가 어디에서 발생하는지 코드에서 확인한다.

- 얼굴이 카메라 경계를 벗어나서 detection 되는 경우에 refined_x가 음수가 되어 발생하는 문제이다. 따라서 음수가 되었을 때 에러가 발생하지 않도록 예외처리를 해준다.

```python
if refined_x < 0:
	img_sticker = img_sticker[:, -refined_x:]
	refined_x = 0
elif refined_x + img_sticker.shape[1] >= img_orig.shape[1]:
	img_sticker = img_sticker[:, :-(img_sticker.shape[1]+refined_x-img_orig.shape[1])]
```

### 3. Out bound 오류(경계 밖으로 대상이 나가서 생기는 오류)를 해결한다.

```python
import dlib
import cv2

def img2sticker(img_orig, img_sticker, detector_hog, landmark_predictor):
    # preprocess
    img_rgb = cv2.cvtColor(img_orig, cv2.COLOR_BGR2RGB)
    
    # detector
    dlib_rects = detector_hog(img_rgb, 0)
    if len(dlib_rects) < 1:
        return img_orig
    
    # landmark
    list_landmarks = []
    for dlib_rect in dlib_rects:
        points = landmark_predictor(img_rgb, dlib_rect)
        list_points = list(map(lambda p: (p.x, p.y), points.parts()))
        list_landmarks.append(list_points)
    
    # head coord
    for dlib_rect, landmark in zip(dlib_rects, list_landmarks):
        x = landmark[30][0] # nose
        y = landmark[30][1] - dlib_rect.width()//2
        w = dlib_rect.width()
        h = dlib_rect.width()
        break
    
    # sticker
    img_sticker = cv2.resize(img_sticker, (w,h), interpolation=cv2.INTER_NEAREST)
    
    refined_x = x - w // 2
    refined_y = y - h
    
    if refined_y < 0:
        img_sticker = img_sticker[-refined_y:]
        refined_y = 0

    if refined_x < 0:
        img_sticker = img_sticker[:, -refined_x:]
        refined_x = 0
    elif refined_x + img_sticker.shape[1] >= img_orig.shape[1]:
        img_sticker = img_sticker[:, :-(img_sticker.shape[1]+refined_x-img_orig.shape[1])]

    img_bgr = img_orig.copy()
    sticker_area = img_bgr[refined_y:refined_y+img_sticker.shape[0], refined_x:refined_x+img_sticker.shape[1]]

    img_bgr[refined_y:refined_y+img_sticker.shape[0], refined_x:refined_x+img_sticker.shape[1]] = \
        cv2.addWeighted(sticker_area, 1.0, img_sticker, 0.7, 0)

    return img_bgr
```

- 위와 같이 수정하였더니 정상 동작하였다.

### 4. 다른 예외는 어떤 것들이 있는지 정의한다. 어떤 것이 문제가 되는지 스스로 정해보자.

- 얼굴의 방향과 거리에 따라 스티커의 크기나 위치를 적절하게 조절해주어야 한다.

## 프로젝트 (4) 스티커앱 분석 - 거리, 인원 수, 각도, 시계열 안정성

### 1. 멀어지는 경우에 왜 스티커앱이 동작하지 않는지 분석한다. detection, landmark, blending 단계 중 무엇이 문제일까?

- dlib detection이 문제이다. 멀어지면 detector_hog 단계에서 bbox가 출력되지 않는다.
- 얼굴을 검출하는 사각형보다 얼굴이 훨씬 작아서 사각형 안에서 얼굴을 검출하지 못하는 것이다.
- 광량이 충분히 확보된다면 멀리 있어도 어느 정도 detection이 가능했다.

```python
# preprocess
img_rgb = cv2.cvtColor(img_orig, cv2.COLOR_BGR2RGB)

# detector
img_rgb_vga = cv2.resize(img_rgb, (640, 360))
dlib_rects = detector_hog(img_rgb_vga, 0)
if len(dlib_rects) < 1:
    return img_orig
```

### 2. detector_hog의 문제를 해결하기 위해, 이미지 피라미드를 조절하여 성능을 향상시키는 간단한 방법이 있다. 이 방법을 활용하여 img2sticker 메소드를 간단히 고쳐보자.

```python
def img2sticker(img_orig, img_sticker, detector_hog, landmark_predictor):
    # preprocess
    img_rgb = cv2.cvtColor(img_orig, cv2.COLOR_BGR2RGB)

    # detector
    img_rgb_vga = cv2.resize(img_rgb, (640, 360))
    dlib_rects = detector_hog(img_rgb_vga, 1) # <- 이미지 피라미드 수 변경
    if len(dlib_rects) < 1:
        return img_orig

    # landmark
    list_landmarks = []
    for dlib_rect in dlib_rects:
        points = landmark_predictor(img_rgb_vga, dlib_rect)
        list_points = list(map(lambda p: (p.x, p.y), points.parts()))
        list_landmarks.append(list_points)
```

### 3. 위에서 새롭게 시도한 방법의 문제점

- 기존 30ms/frame에서 120ms/frame 으로 약 4배 느려짐 → 실시간 구동이 불가능하다.

### 4. 실행시간을 만족할 수 있는 방법을 찾아본다.

- hog 디텍터를 딥러닝 기반 디텍터로 변경할 수 있다. hog 학습 단계에서 다양한 각도에 대한 hog 특징을 모두 추출해서 일반화 하기 어렵기 때문에 딥러닝 기반 검출기의 성능이 훨씬 좋다.

### 5. 인원 수, 각도 등 각 문제에 대해서 1-4번을 반복한다

- 인원 수 한 명 이상일 경우, landmark가 겹치거나 해서 제대로 인식이 되지 않을 수 있다.
- 각도의 경우 landmark를 제대로 검출할 수 없는 각도에서는 인식이 되지 않을 수 있다.
- 모델을 변경하여 인식 문제를 개선했을 때, inference 시간이 증가한다면 사람이 증가할 수록 sticker가 출력되는 속도가 느리거나, 동시에 출력되지 않는 문제가 생길 것 같다.

## 프로젝트 (5) 칼만 필터 적용하기

### 1. 카메라 앞에서 가만히 있을 때 스티커의 움직임을 관찰한다. 어떤 문제가 발생하는가?

- 가만히 있어도 스티커의 크기가 일정하게 유지되지 않고, 떨리는 것처럼 보이는 현상이 발생한다.

### 2. 이론 강의에서 배운 칼만 필터를 적용해서 스티커 움직임을 안정화시킨다.

```python
# webcam_sticker.py
import numpy as np
import cv2
import dlib

from newaddsticker import img2sticker
from kfaddsticker import img2sticker_kf

detector_hog = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor('./models/shape_predictor_68_face_landmarks.dat')

def main():
    cv2.namedWindow('show', 0)
    cv2.resizeWindow('show', 640, 360)

    vc = cv2.VideoCapture(0) # 웹캠 사용으로 변경
    img_sticker = cv2.imread('./images/king.png')

    vlen = int(vc.get(cv2.CAP_PROP_FRAME_COUNT))
    print (vlen) # 웹캠은 video length 가 0 입니다.

    # 정해진 길이가 없기 때문에 while 을 주로 사용합니다.
    # for i in range(vlen):
    while True:
        ret, img = vc.read()
        if ret == False:
            break
        start = cv2.getTickCount()
        img = cv2.flip(img, 1)  # 보통 웹캠은 좌우 반전

        # 칼만 필터 적용
        img_result = img2sticker_kf(img, img_sticker.copy(), detector_hog, landmark_predictor)

        time = (cv2.getTickCount() - start) / cv2.getTickFrequency() * 1000
        print ('[INFO] time: %.2fms'%time)
        
        cv2.imshow('show', img_result)
        key = cv2.waitKey(1)
        if key == 27:
            break


if __name__ == '__main__':
    main()
```

## 회고록

- 웹캠을 이용해 이미지를 처리하는 방법에 대해 궁금했었는데 이번 노드를 통해 알게 되어서 웹캠을 이용한 Object Detection 서비스도 만들어볼 수 있을 것 같다.
- 스티커앱 이외에 다른 필터들도 한번 만들어보고 싶다.